In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
from tqdm import tqdm
from scipy import ndimage

In [ ]:
import torch.nn.functional as F
import torch as tc
from helpers_img import *
from NeuralNets import *
from training_NN import *
from Post_processing import *
from submission import *
from Bagging_Net import *

In [ ]:
tc.cuda.empty_cache()

In [ ]:
# Loaded a set of images
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
N = min(100, len(files)) # Load maximum 100 images
print("Loading " + str(N) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(N)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(N) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(N)]
print(files[0])



In [ ]:
# augment the dataset adding rotated images
imgs, gt_imgs = rotation(imgs, gt_imgs)
print('Total number of images: '+str(len(imgs)))

Create and train the list of models

In [ ]:
percentage_train_data=0.1
nb_model=10
w=16
h=16
lr=1e-4
max_epochs=20
mini_batch_size=10
dropout=0
models, F1_error= bagging_NN(imgs, gt_imgs, percentage_train_data, nb_model, w, h, lr, max_epochs, mini_batch_size, dropout)

Save the models for future needs.

In [ ]:
for i,model in enumerate(models):
    tc.save(model,f'Model_Bagging/model{i}.pt')

Load the models

In [ ]:
models=[]
for i in range(10):
    model = tc.load(f'Model_Bagging/model{i}.pt')
    models.append(model)

Plot the prediction of the models on a image taken from the training set

In [ ]:
test_img = imgs[30]
mask= gt_imgs[30]
test_img = [img_crop(test_img, 16,16)]
test_img = transform_subIMG_to_Tensor(test_img)
mean=test_img.mean()
std= test_img.std()
test_img = (test_img-mean)/std
result=np.zeros((test_img.size(0),))
for model in models:
    result += model(test_img).detach_().numpy().reshape(-1,)

result = result / len(models)
result = (result > 0.5)*1


image = imgs[30]

mask_res = label_to_img(400, 400, 16, 16, result)
image_plot = make_img_overlay(image, mask_res)
plt.figure(figsize=(10, 10))
plt.imshow(image_plot)
    

Load test image, than plot a prediction and finally create the file submission

In [ ]:
# Loaded a set of images
root_dir_test = "test_set_images/"
imgs_test = []
for l in range(1,51):
    dir_test = root_dir_test + 'test_'+str(l)+'/'
    files_test = os.listdir(dir_test)
    img_test = load_image(dir_test + files_test[0])
    imgs_test.append(img_test)

In [ ]:
print_image_test_bagging(imgs_test[16], models)

In [ ]:
create_submission(imgs_test, models, w, h, 'submission_bagging.csv', '', normalize=True)